In [ ]:
import pandas as pd
import numpy as np

# read data from xls file 
df = pd.read_csv('features1.csv', header = None, na_values = '?', index_col = None)

filenames = ['features2.csv', 'features3.csv', 'features4.csv', 'features5.csv', 'features6.csv','test1.csv','test2.csv']
for filename in filenames:
    df1 = pd.read_csv(filename, header = None, na_values = '0', index_col = None)
    df = pd.concat([df,df1], ignore_index = True)
print(np.shape(df))


In [ ]:
# get the value 
y_raw = np.array(df[137])

# list of 2D-features in the order of MFCC, Energy, Pitch
X_raw = np.array(pd.DataFrame(df, columns = df.columns[0:136]))
# print(np.shape(Xtrain))
# print(Xtrain[135])

# scaler = preprocessing.StandardScaler()
# X = scaler.fit_transform(X)

# index = np.random.permutation(1612)
# ntr = 1200
# nts = 412
# # # print(np.shape(X))

# xtr = X[index[0:ntr]]
# ytr = y[index[0:ntr]]
# xts = X[index[ntr:ntr+nts]]
# yts = y[index[ntr:ntr+nts]]


In [ ]:
# one hot coding 
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from collections import Counter

y_reshape = y_raw.reshape(-1, 1)
encoder = preprocessing.OneHotEncoder(sparse=False)
yhot = encoder.fit_transform(y_reshape)
print(yhot)
print(yhot.shape)

In [ ]:
# preprocessing the data
# normalize
N, ydim, xdim = X_raw.shape
X_re = X_raw.reshape(N, xdim*ydim)
X = preprocessing.scale(X_re)
# scaler = preprocessing.standardScaler().fit(X_re)
# X = scaler.transform(X_re)

In [ ]:
test_rate = 0.2
Xtr, Xts, ytr, yts = train_test_split(X, y_raw, test_size=test_rate, shuffle=True, random_state=0)

cnt = Counter(ytr)
print "Number of files in each category in TRAIN set:"
for k in sorted(cnt.keys()):
    print k, ":", cnt[k]

In [ ]:
## use SVM model to predict class

In [ ]:
# svc.fit(Xtr,ytr)

In [ ]:
# yhat_ts = svc.predict(Xts)
# acc = np.mean(yhat_ts == yts)
# print('Accuaracy = {0:f}'.format(acc))

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import precision_recall_fscore_support
# from random import shuffle

# def train_classifier(xtr, ytr):
#     print(xtr.shape)
#     print(ytr.shape)
#     dataset = np.hstack([xtr,xtr])
#     shuffle(dataset)
#     xtr = dataset[0:136]
#     ytr = dataset[137]
#     clf = RandomForestClassifier(n_estimators=1000, max_depth = 100000, n_jobs = -1, oob_score = True, min_samples_split = 10)
#     clf.fit(xtr, ytr)
#     return clf


# def test_classifier(classifier, xts, yts):
#     y_pred = classifier.predict(xts)
#     precision, recall, f1_score, _ = precision_recall_fscore_support(y_ts, y_pred)
#     return [precision, recall, f1_score]



In [ ]:
# clf = train_classifier(xtr,ytr)
# [precision, recall, f1_score] = test_classifier(clf,xts,yts)

In [ ]:
import keras
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Input, Convolution2D, MaxPooling2D, Activation, merge
from sklearn.metrics import accuracy_score


In [ ]:
import keras.backend as K
K.clear_session()

In [ ]:
# try to use pre-trained deep learning network vgg16
Xtr = Xtr.reshape(Xtr.shape[0], ydim, xdim, 1)
Xts = Xts.reshape(Xts.shape[0], ydim, xdim, 1)
Xtr_1 = np.array([Xtr,Xtr,Xtr])
Xts_1 = np.zrray([Xts,Xts,Xts])

pre_trained = 'vgg16'

# Load appropriate packages
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import decode_predictions, preprocess_input    

input_shape = (ydim,xdim,3)
base_model = applications.VGG16(weights='imagenet', include_top = False, input_shape = input_shape)

In [ ]:
model = Sequential()

for layers in base_model.layers:
    model.add(layers)
    
for layers in model.layers:
    layers.trainable = False
    
model.add(Flatten())
model.add(Dense(256,activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation = 'sigmoid'))
model.summary()

In [ ]:
batch_size = 32
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
train_generator = train_datagen.flow(
                        Xtr,ytr
                        target_size=(ydim,xdim),
                        batch_size=batch_size)

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow(
                        Xts,yts
                        target_size=(ydim,xdim), 
                        batch_size=batch_size)

In [ ]:
opt = optimizers.Adam(lr=0.001) # beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
train_size = train_generator.n
test_size = test_generator.n
# print(train_size)
# print(test_size)
steps_per_epoch =  train_size // batch_size
validation_steps =  test_size // batch_size

In [ ]:
nepochs = 5  # Number of epochs

# Call the fit function
model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=nepochs,
    validation_data=test_generator,
    validation_steps=validation_steps)

In [ ]:
## another method use self defined CNN model

In [ ]:
#kares package use Tensorflow as backend
Xtr = Xtr.reshape(Xtr.shape[0], ydim, xdim, 1)
Xts = Xts.reshape(Xts.shape[0], ydim, xdim, 1)
print(Xtr.shape)
print(Xts.shape)
in_shape = Xtr.shape[1:]

In [ ]:
np.random.seed(0) 

model = Sequential()

conv_filters = 16   # number of convolution filters (= CNN depth)
#conv_filters = 32   # number of convolution filters (= CNN depth)

# Layer 1
model.add(Convolution2D(conv_filters, 3, 3, input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2))) 
#model.add(Dropout(0.25)) 

# Layer 2
model.add(Convolution2D(conv_filters, 3, 3))
model.add(MaxPooling2D(pool_size=(2, 2))) 

# After Convolution, we have a 16*x*y matrix output
# In order to feed this to a Full(Dense) layer, we need to flatten all data
# Note: Keras does automatic shape inference, i.e. it knows how many (flat) input units the next layer will need,
# so no parameter is needed for the Flatten() layer.
model.add(Flatten()) 

# Full layer
model.add(Dense(200, activation='sigmoid')) 
#model.add(Dense(256, activation='sigmoid')) 
model.add(Dropout(0.25))

# Output layer
# For binary/2-class problems use ONE sigmoid unit, for multi-class/multi-label problems use n output units 
# activation should be 'softmax' for multi-class / single-label output, 'sigmoid' for binary or multi-label tasks
model.add(Dense(n_classes,activation='softmax'))

In [ ]:
model.summary()

In [ ]:
# Define a loss function 
loss = 'categorical_crossentropy' 

# Optimizer = Stochastic Gradient Descent
optimizer = optimizers.SGD(lr=learn_rate)

# Compiling the model
model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])

In [ ]:
history = None

In [ ]:
# TRAINING the model

# YOU MAY RUN THIS CELL MULTIPLE TIMES TO CONTINUE TO TRAIN THE MODEL FURTHER

# for how many epochs (iterations) to train
epochs = 15

# for training we need the "1 hot encoded" numeric classes of the ground truth
# History = model.fit(train_set, train_classes_1hot, batch_size=32, nb_epoch=epochs)
validation_percent = 0.1
History = model.fit(train_set, train_classes_1hot, validation_split=validation_percent, batch_size=32, nb_epoch=epochs)

# we keep the history of accuracies on training set
# we append this to previous history in case we execute this cell multiple times
if history is None:
    history = History.history
else:
    for key in History.history.keys():
        history[key].extend(History.history[key])

In [ ]:
test_pred = model.predict_classes(test_set)

In [ ]:
accuracy_score(test_classes, test_pred)

In [ ]:
# print(xtr.shape)
# nin = xtr.shape[1]  # dimension of input data
# nh = 100     # number of hidden units
# nout = 8    # number of outputs = 10 since there are 10 classes
# model = Sequential()
# model.add(Dense(nh, input_shape=(nin,), activation='sigmoid', name='hidden'))
# model.add(Dense(nout, activation='softmax', name='output'))
# model.summary()

In [ ]:
# class LossHistory(keras.callbacks.Callback):
#     def on_train_begin(self, logs={}):
#         # TODO:  Create two empty lists, self.loss and self.val_acc
#         self.loss = []
#         self.val_acc = []
 
#     def on_batch_end(self, batch, logs={}):
#         # TODO:  This is called at the end of each batch.  
#         # Add the loss in logs.get('loss') to the loss list
#         loss = logs.get('loss')
#         self.loss.append(loss)
        
#     def on_epoch_end(self, epoch, logs):
#         # TODO:  This is called at the end of each epoch.  
#         # Add the test accuracy in logs.get('loss') to the val_acc list
#         acc = logs.get('val_acc')
#         self.val_acc.append(acc)

# # Create an instance of the history callback
# history_cb = LossHistory()

In [ ]:
# from keras import optimizers
# opt = optimizers.Adam(lr=1e-5) # beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
# model.compile(optimizer=opt,
#               loss='sparse_categorical_crossentropy',
#               metrics=['accuracy'])

In [ ]:
# batch_size = 50
# model.fit(xtr, ytr, epochs=10, batch_size=batch_size, validation_data=(xts,yts), callbacks=[history_cb])